In [19]:
import pandas as pd
import yfinance as yf
from datetime import date

In [20]:
symbol = 'USD'
calendar_days = 174

start_date = '2023-1-1'
trading_days = round(calendar_days * 252 / 365)
sma_days = trading_days
tickerData = yf.Ticker(symbol)
history = tickerData.history(interval='1d', start=start_date, end=date.today())
if symbol == 'USD':
  print('Loading USD history from file')
  history = pd.read_csv('data/USD-daily.csv').set_index('Date')
history['SMA'] = history['Close'].rolling(window=sma_days).mean()
history['SMA_dist'] = history.Close / history.SMA - 1

$USD: possibly delisted; no price data found  (1d 2023-1-1 -> 2025-02-22)


Loading USD history from file


In [22]:
last_sma = history.SMA.iloc[-1]
last_close = history.Close.iloc[-1]

sma_dist = history.SMA_dist.dropna()
sma_range = (sma_dist <= sma_dist.iloc[-1]).mean()

print(symbol)
print(f'Start:    {history.index[0]}')
print(f'Price:    ${last_close:.2f} (P{sma_range:.0%})')
print(f'SMA {sma_days:3d}d: ${last_sma:.2f}')

for q in [0.5, 0.3, 0.25, 0.1, 0.05]:
  dist = sma_dist.quantile(q)
  price = last_sma * (1 + dist)
  print(f'P   {q:4.0%}: ${price:5.2f} (SMA{sma_days}{dist:+.1%})')


USD
Start:    1/3/2023 16:00:00
Price:    $62.81 (P14%)
SMA 120d: $63.76
P    50%: $73.63 (SMA120+15.5%)
P    30%: $67.87 (SMA120+6.4%)
P    25%: $66.56 (SMA120+4.4%)
P    10%: $61.34 (SMA120-3.8%)
P     5%: $56.93 (SMA120-10.7%)
